# Import Library

In [1]:
import pandas as pd
import duckdb
import os

In [2]:
import duckdb
print(duckdb.__version__)


1.0.0


# Make eklipse_data.db

In [3]:
# Inisialisasi koneksi DuckDB (in-memory atau file)
con = duckdb.connect(database=':memory:')  # Atau ganti dengan 'database.duckdb' kalau ingin persist

# Load semua CSV sebagai view (DuckDB bisa baca langsung dari disk!)
con.execute("""
    CREATE VIEW downloaded_clips AS SELECT * FROM read_csv_auto('da_test/downloaded_clips.csv');
    CREATE VIEW shared_clips AS SELECT * FROM read_csv_auto('da_test/shared_clips.csv');
    CREATE VIEW clips AS SELECT * FROM read_csv_auto('da_test/clips.csv');
    CREATE VIEW premium_users AS SELECT * FROM read_csv_auto('da_test/premium_users.csv');
    CREATE VIEW gamesession AS SELECT * FROM read_csv_auto('da_test/gamesession.csv');
""")

## SQL

## Question SQL 1
The total number of users who have downloaded any clips. total number of clips downloaded. The total number of game sessions from which the downloaded clips were generated. 

In [4]:
query = """--sql 
SELECT
    COUNT(DISTINCT user_id) AS total_user_who_download_clips,
    COUNT(DISTINCT clip_id) AS total_clips_downloaded,
    COUNT(DISTINCT gamesession_id) AS total_gamesession_generated
FROM downloaded_clips dc;"""

df = con.execute(query).fetchdf()
df


,total_user_who_download_clips,total_clips_downloaded,total_gamesession_generated
0,1454,12935,4338


## Question SQL 2
For users who purchased premium in the last 3 months: Get the number of 
users who purchased premium, the number of users who shared any clips, 
the total number of clips shared, and the total number of gamesessions from 
which the shared clips were generated. 

In [5]:
query = """--sql 
WITH premiums_users_last_3_month AS (
    SELECT user_id
    FROM premium_users
    WHERE CAST(starts_at AS TIMESTAMP) >= (
        SELECT MAX(CAST(starts_at AS TIMESTAMP)) - INTERVAL '3 MONTH'
        FROM premium_users
    )
),
premium_summary AS (
    SELECT 
        COUNT(DISTINCT user_id) AS total_user_who_purchased_premium_last_3_months
    FROM premiums_users_last_3_month
),
shared_clips_summary AS (
    SELECT 
        COUNT(DISTINCT sc.user_id) AS number_of_user_premium_last_3_month_who_shared_clips,
        COUNT(DISTINCT sc.clip_id) AS number_of_clip_shared,
        COUNT(DISTINCT sc.gamesession_id) AS number_of_gamesession_generated_from_shared_clips
    FROM shared_clips sc
    JOIN premiums_users_last_3_month pul3m ON sc.user_id = pul3m.user_id
)

SELECT 
    ps.total_user_who_purchased_premium_last_3_months,
    scs.number_of_user_premium_last_3_month_who_shared_clips,
    scs.number_of_clip_shared,
    scs.number_of_gamesession_generated_from_shared_clips
FROM premium_summary ps
CROSS JOIN shared_clips_summary scs;

"""

df = con.execute(query).fetchdf()
df


,total_user_who_purchased_premium_last_3_months,number_of_user_premium_last_3_month_who_shared_clips,number_of_clip_shared,number_of_gamesession_generated_from_shared_clips
0,164,59,1144,359


## Question SQL 3
Calculate, on a weekly basis: The number of users engaged. The number of 
clips engaged. The total number of gamesessions from which the engaged 
clips were generated.

In [6]:
query = """--sql 
WITH premiums_users_last_3_month AS (
    SELECT user_id
    FROM premium_users
    WHERE starts_at >= (
        SELECT MAX(starts_at) - INTERVAL '3 MONTH'
        FROM premium_users
    )
),
premium_summary AS (
    SELECT 
        COUNT(DISTINCT user_id) AS total_user_who_purchased_premium_last_3_months
    FROM premiums_users_last_3_month
),
shared_clips_summary AS (
    SELECT 
        COUNT(DISTINCT sc.user_id) AS number_of_user_premium_last_3_month_who_shared_clips,
        COUNT(DISTINCT sc.clip_id) AS number_of_clip_shared,
        COUNT(DISTINCT sc.gamesession_id) AS number_of_gamesession_generated_from_shared_clips
    FROM shared_clips sc
    JOIN premiums_users_last_3_month pul3m ON sc.user_id = pul3m.user_id
)

SELECT 
    ps.total_user_who_purchased_premium_last_3_months,
    scs.number_of_user_premium_last_3_month_who_shared_clips,
    scs.number_of_clip_shared,
    scs.number_of_gamesession_generated_from_shared_clips
FROM premium_summary ps
CROSS JOIN shared_clips_summary scs;
"""

df = con.execute(query).fetchdf()
df


,total_user_who_purchased_premium_last_3_months,number_of_user_premium_last_3_month_who_shared_clips,number_of_clip_shared,number_of_gamesession_generated_from_shared_clips
0,164,59,1144,359


# B. Data Analysis

In [7]:
query_gamesession_premium_users = """--sql
-----------------------------------------------------------
-- query premium user ids
with gamession_with_premium_users as (
-- query gamesession who the user_id is premium
SELECT 
    *
FROM
    gamesession
WHERE
    user_id IN (SELECT DISTINCT user_id from premium_users)
)

select 
    count(distinct id) as total_gamesession_generated_by_premium_users,
    count(distinct user_id) as total_premium_users_who_generated_gamesession,
    round(avg(duration)/1000,2) as average_duration_gamesession_per_premium_users,
    count(distinct game_name) as total_games_used_by_premium_users_to_process_gamesession  

from 
    gamession_with_premium_users
-----------------------------------------------------------
"""

query_gamesession_free_users = """--sql
-----------------------------------------------------------
-- query free user ids
with gamession_with_free_users as (
-- query gamesession who the user_id is free
SELECT 
    *
FROM
    gamesession
WHERE
    user_id NOT IN (SELECT DISTINCT user_id from premium_users)
)

select 
    count(distinct id) as total_gamesession_generated_by_free_users,
    count(distinct user_id) as total_free_users_who_generated_gamesession,
    round(avg(duration)/1000,2) as average_duration_gamesession_per_free_users,
    count(distinct game_name) as total_games_used_by_free_users_to_process_gamesession
from 
    gamession_with_free_users
-----------------------------------------------------------
"""

df_premium_users = con.execute(query_gamesession_premium_users).fetchdf()
df_free_users = con.execute(query_gamesession_free_users).fetchdf()

df_free_users

,total_gamesession_generated_by_free_users,total_free_users_who_generated_gamesession,average_duration_gamesession_per_free_users,total_games_used_by_free_users_to_process_gamesession
0,28524,4002,6959.48,57


## Gamessesion

### Compare free and premium users based on the gamesessions created by users


In [8]:
query = """--sql
----------------------------------------------------------------------------------------------
with gamesession_with_user_type as (
SELECT 
    g.*,
    CASE 
        WHEN g.user_id IN (SELECT DISTINCT user_id FROM premium_users) THEN 'premium'
        ELSE 'free'
    END AS user_type
FROM 
    gamesession g
)

select
    user_type, 
    count(distinct id) as total_gamesession_generated,
    count(distinct user_id) as total_users_who_generated_gamesession,
    round(avg(duration),0) as average_duration_gamesession,
    count(distinct game_name) as total_games_used_to_process_gamesession,
    
from 
    gamesession_with_user_type
group by user_type
----------------------------------------------------------------------------------------------
"""
df_agg_gamession_by_user_type = con.execute(query).fetchdf()


df_agg_gamession_by_user_type

,user_type,total_gamesession_generated,total_users_who_generated_gamesession,average_duration_gamesession,total_games_used_to_process_gamesession
0,premium,9055,320,10506121.0,53
1,free,28524,4002,6959485.0,57


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Buat subplot 2x2
fig = make_subplots(rows=2, cols=2, subplot_titles=[
    "Total GameSessions",
    "Total Users",
    "Average Duration (sec)",
    "Total Games Used"
])

# Bar chart 1: Total GameSessions
fig.add_trace(go.Bar(
    x=df_agg_gamession_by_user_type['user_type'],
    y=df_agg_gamession_by_user_type['total_gamesession_generated'],
    name='Total GameSessions',
    marker_color='indianred'
), row=1, col=1)

# Bar chart 2: Total Users
fig.add_trace(go.Bar(
    x=df_agg_gamession_by_user_type['user_type'],
    y=df_agg_gamession_by_user_type['total_users_who_generated_gamesession'],
    name='Total Users',
    marker_color='lightsalmon'
), row=1, col=2)

# Bar chart 3: Average Duration
fig.add_trace(go.Bar(
    x=df_agg_gamession_by_user_type['user_type'],
    y=df_agg_gamession_by_user_type['average_duration_gamesession'],
    name='Avg Duration',
    marker_color='dodgerblue'
), row=2, col=1)

# Bar chart 4: Total Games Used
fig.add_trace(go.Bar(
    x=df_agg_gamession_by_user_type['user_type'],
    y=df_agg_gamession_by_user_type['total_games_used_to_process_gamesession'],
    name='Games Used',
    marker_color='mediumseagreen'
), row=2, col=2)

# Update layout
fig.update_layout(
    title_text="Comparison Between Free vs Premium Users",
    showlegend=False,
    height=600,
    width=900
)

fig.show()


We can observe that the total number of gamesessions created and the number of games played are higher among free users compared to premium users. This is expected, as the number of free users who create gamesessions is significantly greater than that of premium users. However, an interesting insight is that premium users tend to create gamesessions with much longer durations compared to free users.

## Compare free and premium users based on the gamesessions created by users for each game.

In [10]:
query = """--sql
WITH gamesession_with_user_type AS (
    SELECT 
        g.*,
        CASE 
            WHEN g.user_id IN (SELECT DISTINCT user_id FROM premium_users) THEN 'premium'
            ELSE 'free'
        END AS user_type
    FROM 
        gamesession g
),
agg AS (
    SELECT 
        user_type, 
        game_name,
        COUNT(DISTINCT id) AS total_gamesession_generated,
        COUNT(DISTINCT user_id) AS total_users_who_generated_gamesession,
        ROUND(AVG(duration), 0) AS average_duration_gamesession
    FROM gamesession_with_user_type
    GROUP BY user_type, game_name
)

SELECT 
    *,
    ROUND(
        total_gamesession_generated * 100.0 
        / SUM(total_gamesession_generated) OVER (PARTITION BY user_type), 
        2
    ) AS pct_gamesession_within_user_type,
    ROUND(
        total_users_who_generated_gamesession * 100.0 
        / SUM(total_users_who_generated_gamesession) OVER (PARTITION BY user_type), 
        2
    ) AS pct_users_within_user_type
FROM agg
ORDER BY user_type, pct_gamesession_within_user_type DESC


---order by user_type, total_gamesession_generated DESC
"""

df_agg_gamession_by_user_type_per_game = con.execute(query).fetchdf()
df_agg_gamession_by_user_type_per_game.head(5)

,user_type,game_name,total_gamesession_generated,total_users_who_generated_gamesession,average_duration_gamesession,pct_gamesession_within_user_type,pct_users_within_user_type
0,free,Other Games,10938,2233,7562694.0,38.35,32.11
1,free,COD: Warzone3-2 & MW3-2,5655,983,6154490.0,19.83,14.14
2,free,Fortnite,3213,807,6394120.0,11.26,11.60
3,free,Valorant,1977,388,8528200.0,6.93,5.58
4,free,Apex Legends,1413,303,7047973.0,4.95,4.36


In [22]:
import plotly.express as px

df_filtered = df_agg_gamession_by_user_type_per_game[~df_agg_gamession_by_user_type_per_game['game_name'].isnull()]


fig = px.bar(
    df_filtered,
    x='pct_gamesession_within_user_type',
    y='game_name',
    color='user_type',
    orientation='h',
    text='pct_gamesession_within_user_type',
    barmode='group',
    title='Percent of Total GameSessions per Game (Free vs Premium) - Horizontal',
    labels={
        "pct_gamesession_within_user_type": "Percent of Total GameSessions",
        "game_name": "Game",
        "user_type": "User Type"
    }
)

fig.update_traces(textposition='outside')
fig.update_layout(
    height=800,
    width=1000,
    margin=dict(l=150, t=80),
    yaxis=dict(categoryorder='total ascending') 
)

fig.show()

######################################################################################
fig = px.bar(
    df_filtered,
    x='pct_users_within_user_type',
    y='game_name',
    color='user_type',
    orientation='h',
    text='pct_users_within_user_type',
    barmode='group',
    title='Percent of Total Users who Generated GameSessions per Game (Free vs Premium) - Horizontal',
    labels={
        "pct_users_within_user_type": "Percent of Total Users",
        "game_name": "Game",
        "user_type": "User Type"
    }
)

fig.update_traces(textposition='outside')
fig.update_layout(
    height=800,
    width=1000,
    margin=dict(l=150, t=80),
    yaxis=dict(categoryorder='total ascending') 
)

fig.show()

######################################################################################
fig = px.bar(
    df_filtered,
    x='average_duration_gamesession',
    y='game_name',
    color='user_type',
    orientation='h',
    text='average_duration_gamesession',
    barmode='group',
    title='Average Duration of GameSessions per Game (Free vs Premium) - Horizontal',
    labels={
        "average_duration_gamesession": "Duration",
        "game_name": "Game",
        "user_type": "User Type"
    }
)

fig.update_traces(textposition='outside')
fig.update_layout(
    height=800,
    width=1000,
    margin=dict(l=150, t=80),
    yaxis=dict(categoryorder='total ascending') 
)

fig.show()
